# **Signal To Spike Conversion** - Baseline Detection
This notebook presents the **baseline detection** step that needs to be conducted before doing the signal-to-spike conversion.

The signal-to-spike conversion requires the definition of the `up_threshold` and `down_threshold` values that will be used to translate the continuous signal into discrete events. Here, we follow the approach proposed by Indiveri's team to define these thresholds. Which is composed of the following steps:
1. Select a time window of the signal representative of the whole signal.
   - The source work used a 1s window.
   - We will test different window sizes.
2. Store the maximum signal amplitudes of consecutive non-overlapping time sub-windows of a defined size.
3. Take the mean of the lowest quartile as the baseline amplitude (or other metric)
   - Check why we are using the lowest quartile.    

### Check WD (change if necessary) and file loading

In [1]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/hfo/signal_to_spike" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/hfo/signal_to_spike"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

/home/monkin/Desktop/feup/thesis
File Location:  /home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/signal_to_spike
New Working Directory:  /home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/signal_to_spike


## Let's first load the filtered Subset SEEG signal
This notebook only converts 1 SEEG channel to spikes. Therefore, the `.npy` file must contain a single channel.

### Declare the `INPUT_FOLDER` and `RESULTS_FOLDER`

In [2]:
# CAREFUL WITH THIS FOLDER TO NOT OVERWRITE THE FILES
DATASET_FILENAME = "seeg_filtered_subset_90-119_segment500_200"
INPUT_FOLDER = f"../subset/results/{DATASET_FILENAME}"
RESULTS_FOLDER = "baseline_results"

### Load the Ripple, Fast Ripple, and Annotated Events

In [3]:
import numpy as np
import math
from utils.io import preview_np_array
from utils.input import RIPPLE_BAND_FILENAME, FR_BAND_FILENAME, BOTH_BAND_FILENAME

# Load the filtered seeg signal in the ripple band
ripple_seeg_file_name = f"{INPUT_FOLDER}/{RIPPLE_BAND_FILENAME}_band.npy"
ripple_band_seeg = np.load(f"{ripple_seeg_file_name}")

# Remove the extra inner dimension
ripple_band_seeg = np.squeeze(ripple_band_seeg)

preview_np_array(ripple_band_seeg, "Ripple Band SEEG", edge_items=3)

Ripple Band SEEG Shape: (245760,).
Preview: [ 1.92911688e-06  2.64027885e-05  1.68256904e-04 ... -9.10608435e-01
 -6.95654667e-01 -4.24182186e-01]


In [4]:
# Load the filtered seeg signal in the fast ripple band
fr_seeg_file_name = f"{INPUT_FOLDER}/{FR_BAND_FILENAME}_band.npy"
fr_band_seeg = np.load(f"{fr_seeg_file_name}")

# Remove the extra inner dimension
fr_band_seeg = np.squeeze(fr_band_seeg)

preview_np_array(fr_band_seeg, "FR Band SEEG", edge_items=3)

FR Band SEEG Shape: (245760,).
Preview: [-3.37759792e-05 -2.44716916e-04 -4.38382591e-04 ... -1.90716295e-01
 -1.66433282e-01 -5.55778231e-02]


In [5]:
# TODO: Baseline Detection for the HFO Band (Ripple and FR)
# Load the filtered seeg signal in the HFO band
hfo_seeg_file_name = f"{INPUT_FOLDER}/{BOTH_BAND_FILENAME}_band.npy"
hfo_band_seeg = np.load(f"{hfo_seeg_file_name}")

# Remove the extra inner dimension
hfo_band_seeg = np.squeeze(hfo_band_seeg)

preview_np_array(hfo_band_seeg, "HFO Band SEEG", edge_items=3)

HFO Band SEEG Shape: (245760,).
Preview: [ 0.00164447  0.01391946  0.04441551 ... -0.97813142 -1.05971408
 -1.08183859]


In [6]:
# Load the annotated events (For Ripples)
ripple_markers_file_name = f"{INPUT_FOLDER}/markers_{RIPPLE_BAND_FILENAME}_band.npy"
ripple_markers = np.load(f"{ripple_markers_file_name}")

# Remove the extra inner dimension
ripple_markers = np.squeeze(ripple_markers)

preview_np_array(ripple_markers, "ripple_markers", edge_items=3)

ripple_markers Shape: (197,).
Preview: [('Ripple+Fast-Ripple',   1000.  , 0.)
 ('Spike+Ripple+Fast-Ripple',   3206.54, 0.)
 ('Spike+Ripple',   3521.  , 0.) ... ('Ripple', 116037.  , 0.)
 ('Ripple', 116721.  , 0.) ('Spike+Ripple+Fast-Ripple', 119000.  , 0.)]


In [7]:
# Load the annotated events (For Fast Ripples)
fr_markers_file_name = f"{INPUT_FOLDER}/markers_{FR_BAND_FILENAME}_band.npy"
fr_markers = np.load(f"{fr_markers_file_name}")

# Remove the extra inner dimension
fr_markers = np.squeeze(fr_markers)

preview_np_array(fr_markers, "fr_markers", edge_items=3)

fr_markers Shape: (199,).
Preview: [('Fast-Ripple',   1000.  , 0.)
 ('Spike+Ripple+Fast-Ripple',   3206.54, 0.)
 ('Fast-Ripple',   3770.02, 0.) ... ('Fast-Ripple', 116096.  , 0.)
 ('Ripple+Fast-Ripple', 116656.  , 0.)
 ('Spike+Ripple+Fast-Ripple', 119000.  , 0.)]


In [8]:
# Load the annotated events (For HFOs - Both Ripple and FR)
hfo_markers_file_name = f"{INPUT_FOLDER}/markers_{BOTH_BAND_FILENAME}_band.npy"
hfo_markers = np.load(f"{hfo_markers_file_name}")

# Remove the extra inner dimension
hfo_markers = np.squeeze(hfo_markers)

preview_np_array(hfo_markers, "hfo_markers", edge_items=3)

hfo_markers Shape: (222,).
Preview: [('Ripple+Fast-Ripple',   1000.  , 0.)
 ('Spike+Ripple+Fast-Ripple',   3206.54, 0.)
 ('Spike+Ripple',   3521.  , 0.) ... ('Ripple', 116037.  , 0.)
 ('Ripple', 116721.  , 0.) ('Spike+Fast-Ripple', 119000.  , 0.)]


There are 197 Ripple Events, 199 Fast Ripple Events and 222 HFO Events in the dataset for the whole time. Since we are considering the whole recording, that means all the events are present during the window. Thus, we can **expect to have around 200 Events** in the window. -> Relevant Event Rate ~= 200 / 120s = 1.67 Events/s

Therefore, there is an event ~ every `1/1.67s` ~= `0.6s`.

We can use a `sub_window_size` of `0.6/4` = `0.15s`. This way, about 25% of the sub-windows will contain a relevant event. This is useful to decide the heuristic that will select the baseline threshold. An intuitive one is to select the Q3 (75th percentile) of the maximum amplitudes of the sub-windows.

## 1. Select a time window of the signal representative of the whole signal

In [9]:
import math
from utils.input import SAMPLING_RATE, X_STEP

window_size = 120 * (10**3)  # 60 seconds or 60,000 ms

# Crop the signal to the respective window_size
# find the last index before the window_size
num_samples_window = math.ceil(window_size / X_STEP)

# Crop the signal to the respective window_size
ripple_band_seeg = ripple_band_seeg[:num_samples_window]
fr_band_seeg = fr_band_seeg[:num_samples_window]
hfo_band_seeg = hfo_band_seeg[:num_samples_window]

preview_np_array(ripple_band_seeg, "Cropped Ripple Band SEEG", edge_items=3)
preview_np_array(fr_band_seeg, "Cropped FR Band SEEG", edge_items=3)
preview_np_array(hfo_band_seeg, "Cropped HFO Band SEEG", edge_items=3)

Cropped Ripple Band SEEG Shape: (245760,).
Preview: [ 1.92911688e-06  2.64027885e-05  1.68256904e-04 ... -9.10608435e-01
 -6.95654667e-01 -4.24182186e-01]
Cropped FR Band SEEG Shape: (245760,).
Preview: [-3.37759792e-05 -2.44716916e-04 -4.38382591e-04 ... -1.90716295e-01
 -1.66433282e-01 -5.55778231e-02]
Cropped HFO Band SEEG Shape: (245760,).
Preview: [ 0.00164447  0.01391946  0.04441551 ... -0.97813142 -1.05971408
 -1.08183859]


## Store the maximum signal amplitudes of consecutive non-overlapping time sub-windows
For this, we need to define the length of the `sub_windows`.

Since we floor the value, some values in the end of the window will not be considered if it is not a multiple of the sub-window length

In [10]:
# Each sub-window is 0.15s
sub_window_scale = 15/100
sub_window_length = math.floor((num_samples_window / 120) * sub_window_scale)   # Define the sub-window length according to the window size 
print("Sub-Window Length: ", sub_window_length)

Sub-Window Length:  307


### Iterate over the sub-windows and store the maximum/minimum signal amplitude of each sub-window

In [11]:
ripple_max_amplitudes = []
ripple_min_amplitudes = []
fr_max_amplitudes = []
fr_min_amplitudes = []
hfo_max_amplitudes = []
hfo_min_amplitudes = []

for start_idx in range(0, num_samples_window - sub_window_length, sub_window_length):
    end_idx = start_idx + sub_window_length
    # Crop the signal to the respective window_size
    ripple_sub_window_seeg = ripple_band_seeg[start_idx:end_idx]
    fr_sub_window_seeg = fr_band_seeg[start_idx:end_idx]
    hfo_sub_window_seeg = hfo_band_seeg[start_idx:end_idx]

    # Get the maximum signal amplitude in the sub-window
    ripple_max_amp = np.max(ripple_sub_window_seeg)
    fr_max_amp = np.max(fr_sub_window_seeg)
    hfo_max_amp = np.max(hfo_sub_window_seeg)
    # Get the minimum signal amplitude in the sub-window
    ripple_min_amp = np.min(ripple_sub_window_seeg)
    fr_min_amp = np.min(fr_sub_window_seeg)
    hfo_min_amp = np.min(hfo_sub_window_seeg)

    # Append the maximum amplitudes to the respective lists
    ripple_max_amplitudes.append(ripple_max_amp)
    fr_max_amplitudes.append(fr_max_amp)
    hfo_max_amplitudes.append(hfo_max_amp)
    # Append the minimum amplitudes to the respective lists
    ripple_min_amplitudes.append(ripple_min_amp)
    fr_min_amplitudes.append(fr_min_amp)
    hfo_min_amplitudes.append(hfo_min_amp)

ripple_max_amplitudes = np.array(ripple_max_amplitudes)
fr_max_amplitudes = np.array(fr_max_amplitudes)
hfo_max_amplitudes = np.array(hfo_max_amplitudes)
ripple_min_amplitudes = np.array(ripple_min_amplitudes)
fr_min_amplitudes = np.array(fr_min_amplitudes)
hfo_min_amplitudes = np.array(hfo_min_amplitudes)

preview_np_array(ripple_max_amplitudes, "Ripple Max Amplitudes", edge_items=3)
preview_np_array(ripple_min_amplitudes, "Ripple Min Amplitudes", edge_items=3)
preview_np_array(fr_max_amplitudes, "FR Max Amplitudes", edge_items=3)
preview_np_array(fr_min_amplitudes, "FR Min Amplitudes", edge_items=3)
preview_np_array(hfo_max_amplitudes, "HFO Max Amplitudes", edge_items=3)
preview_np_array(hfo_min_amplitudes, "HFO Min Amplitudes", edge_items=3)

num_sub_windows = len(ripple_max_amplitudes)

Ripple Max Amplitudes Shape: (800,).
Preview: [2.339884   2.77546019 2.11573086 ... 3.04313005 1.91448754 1.50260109]
Ripple Min Amplitudes Shape: (800,).
Preview: [-1.71832736 -2.4413664  -1.97949198 ... -1.72345595 -2.37643696
 -1.82233626]
FR Max Amplitudes Shape: (800,).
Preview: [1.14012585 1.43422742 1.48679787 ... 1.26333557 1.10133063 1.30209737]
FR Min Amplitudes Shape: (800,).
Preview: [-1.24652631 -1.33902546 -1.53658026 ... -1.22057082 -1.1672457
 -1.40985366]
HFO Max Amplitudes Shape: (800,).
Preview: [2.89055003 3.00696624 3.03855587 ... 3.17677869 2.52820922 2.62339345]
HFO Min Amplitudes Shape: (800,).
Preview: [-2.25665024 -3.19481751 -2.36174023 ... -2.54457291 -2.75380369
 -1.73551577]


### Plot the maximum signal amplitudes of the sub-windows as an Histogram

Bins for the Ripple Band

In [12]:
# Plot the maximum amplitudes of the sub-windows in a histogram
from utils.bar_plot import create_histogram  # Import the function to create the figure

# Define the histogram bins
ripple_min_amp = np.min(ripple_max_amplitudes)
ripple_max_amp = np.max(ripple_max_amplitudes)
bins = np.linspace(ripple_min_amp, ripple_max_amp, 100)    

ripple_max_amp_hist = create_histogram(
    title="Ripple Sub-Window Max. Amplitude Histogram", 
    x_axis_label='Amplitude (uV)', 
    y_axis_label='Count',
    bins=bins,
    x=ripple_max_amplitudes,
    is_density=False,
    sizing_mode="stretch_width",
)

In [13]:
# Plot the minimum amplitudes of the sub-windows in a histogram
# Define the histogram bins
ripple_min_amp = np.min(ripple_min_amplitudes)
ripple_max_amp = np.max(ripple_min_amplitudes)
bins = np.linspace(ripple_min_amp, ripple_max_amp, 100)    

ripple_min_amp_hist = create_histogram(
    title="Ripple Sub-Window Min. Amplitude Histogram", 
    x_axis_label='Amplitude (uV)', 
    y_axis_label='Count',
    bins=bins,
    x=ripple_min_amplitudes,
    is_density=False,
    sizing_mode="stretch_width",
)

Bins for the FR Band

In [14]:
# Define the histogram bins
fr_min_amp = np.min(fr_max_amplitudes)
fr_max_amp = np.max(fr_max_amplitudes)
bins = np.linspace(fr_min_amp, fr_max_amp, 100)    

fr_max_amp_hist = create_histogram(
    title="FR Sub-Window Max. Amplitude Histogram", 
    x_axis_label='Amplitude (uV)', 
    y_axis_label='Count',
    bins=bins,
    x=fr_max_amplitudes,
    is_density=False,
    sizing_mode="stretch_width",
)

In [15]:
# Define the histogram bins
fr_min_amp = np.min(fr_min_amplitudes)
fr_max_amp = np.max(fr_min_amplitudes)
bins = np.linspace(fr_min_amp, fr_max_amp, 100)    

fr_min_amp_hist = create_histogram(
    title="FR Sub-Window Min. Amplitude Histogram", 
    x_axis_label='Amplitude (uV)', 
    y_axis_label='Count',
    bins=bins,
    x=fr_min_amplitudes,
    is_density=False,
    sizing_mode="stretch_width",
)

Bins for the HFO Band (Ripple + FR)

In [16]:
# Define the histogram bins
hfo_min_amp = np.min(hfo_max_amplitudes)
hfo_max_amp = np.max(hfo_max_amplitudes)
bins = np.linspace(hfo_min_amp, hfo_max_amp, 100)    

hfo_max_amp_hist = create_histogram(
    title="HFO Sub-Window Max. Amplitude Histogram", 
    x_axis_label='Amplitude (uV)', 
    y_axis_label='Count',
    bins=bins,
    x=hfo_max_amplitudes,
    is_density=False,
    sizing_mode="stretch_width",
)

In [17]:
# Define the histogram bins
hfo_min_amp = np.min(hfo_min_amplitudes)
hfo_max_amp = np.max(hfo_min_amplitudes)
bins = np.linspace(hfo_min_amp, hfo_max_amp, 100)    

hfo_min_amp_hist = create_histogram(
    title="HFO Sub-Window Min. Amplitude Histogram", 
    x_axis_label='Amplitude (uV)', 
    y_axis_label='Count',
    bins=bins,
    x=hfo_min_amplitudes,
    is_density=False,
    sizing_mode="stretch_width",
)

In [18]:
import bokeh.plotting as bplt

show_ripple_hist_plot = True
if show_ripple_hist_plot:
    # Show the plots
    bplt.show(ripple_min_amp_hist)
    bplt.reset_output()

    bplt.show(ripple_max_amp_hist)
    # Close the plot
    bplt.reset_output()

In [59]:
show_fr_hist_plot = True
if show_fr_hist_plot:
    # Show the plot
    bplt.show(fr_min_amp_hist)
    bplt.reset_output()

    bplt.show(fr_max_amp_hist)
    bplt.reset_output()

In [60]:
show_hfo_hist_plot = True
if show_hfo_hist_plot:
    # Show the plot
    bplt.show(hfo_min_amp_hist)
    bplt.reset_output()

    bplt.show(hfo_max_amp_hist)
    bplt.reset_output()

### Plot the maximum signal amplitudes of the sub-windows as a Bar Plot

In [61]:
from utils.bar_plot import create_box_plot

# Create the BoxPlot
ripple_max_amp_boxplot = create_box_plot(
    title="Boxplot of Ripple Sub-Window Max. Amplitudes", 
    box_arrays=[ripple_max_amplitudes],
    y_axis_label='Amplitude (uV)',
    sizing_mode="stretch_width",
)

show_ripple_barplot = True
if show_ripple_barplot:
    # Show the plot
    bplt.show(ripple_max_amp_boxplot)

ripple_quantiles:  [2.1719012  2.67741361 9.42601347]
Ripple IQR:  7.25411226416056


In [62]:
# Create the BoxPlot
fr_max_amp_boxplot = create_box_plot(
    title="Boxplot of Fast Ripple Sub-Window Max. Amplitudes", 
    box_arrays=[fr_max_amplitudes],
    y_axis_label='Amplitude (uV)',
    sizing_mode="stretch_width",
)

show_fr_barplot = True
if show_fr_barplot:
    # Show the plot
    bplt.show(fr_max_amp_boxplot)

# TODO: Not plotting the boxplot for the minimum amplitudes

ripple_quantiles:  [1.17348301 1.37531467 5.34708415]
Ripple IQR:  4.173601136673139


In [63]:
# Create the BoxPlot
hfo_max_amp_boxplot = create_box_plot(
    title="Boxplot of HFO Sub-Window Max. Amplitudes", 
    box_arrays=[hfo_max_amplitudes],
    y_axis_label='Amplitude (uV)',
    sizing_mode="stretch_width",
)

show_hfo_barplot = True
if show_hfo_barplot:
    # Show the plot
    bplt.show(hfo_max_amp_boxplot)

# TODO: Not plotting the boxplot for the minimum amplitudes

ripple_quantiles:  [2.54246975 3.09298839 8.58666474]
Ripple IQR:  6.044194981642361


In [64]:
# Create the BoxPlot
combined_max_amp_boxplot = create_box_plot(
    title="Boxplot of Sub-Window Max. Amplitudes for different frequency bands", 
    box_arrays=[ripple_max_amplitudes, fr_max_amplitudes, hfo_max_amplitudes],
    y_axis_label='Amplitude (uV)',
    sizing_mode="stretch_width",
)

show_combined_barplot = True
if show_combined_barplot:
    # Show the plot
    bplt.show(combined_max_amp_boxplot)

# TODO: Not plotting the boxplot for the minimum amplitudes

ripple_quantiles:  [2.1719012  2.67741361 9.42601347]
Ripple IQR:  7.25411226416056
ripple_quantiles:  [1.17348301 1.37531467 5.34708415]
Ripple IQR:  4.173601136673139
ripple_quantiles:  [2.54246975 3.09298839 8.58666474]
Ripple IQR:  6.044194981642361


### Show some statistics of the maximum sub-window amplitudes

In [19]:
mean_max_ripple_amp = np.mean(ripple_max_amplitudes)
std_max_ripple_amp = np.std(ripple_max_amplitudes)
median_max_ripple_amp = np.median(ripple_max_amplitudes)
print("Mean Max. Ripple Amplitude: ", mean_max_ripple_amp)
print("Std Max. Ripple Amplitude: ", std_max_ripple_amp)
print("Median Max. Ripple Amplitude: ", median_max_ripple_amp)

Mean Max. Ripple Amplitude:  6.128352043987768
Std Max. Ripple Amplitude:  6.644250044979027
Median Max. Ripple Amplitude:  2.6774136055947038


In [20]:
mean_max_fr_amp = np.mean(fr_max_amplitudes)
std_max_fr_amp = np.std(fr_max_amplitudes)
median_max_fr_amp = np.median(fr_max_amplitudes)
print("Mean Max. FR Amplitude: ", mean_max_fr_amp)
print("Std Max. FR Amplitude: ", std_max_fr_amp)
print("Median Max. FR Amplitude: ", median_max_fr_amp)

Mean Max. FR Amplitude:  2.7920384114064505
Std Max. FR Amplitude:  2.4595597037981696
Median Max. FR Amplitude:  1.3753146660939348


In [21]:
mean_max_hfo_amp = np.mean(hfo_max_amplitudes)
std_max_hfo_amp = np.std(hfo_max_amplitudes)
median_max_hfo_amp = np.median(hfo_max_amplitudes)
print("Mean Max. HFO Amplitude: ", mean_max_hfo_amp)
print("Std Max. HFO Amplitude: ", std_max_hfo_amp)
print("Median Max. HFO Amplitude: ", median_max_hfo_amp)

Mean Max. HFO Amplitude:  6.317369652901671
Std Max. HFO Amplitude:  6.762035632847155
Median Max. HFO Amplitude:  3.0929883901351527


### Show some statistics of the minimum sub-window amplitudes

In [22]:
mean_min_ripple_amp = np.mean(ripple_min_amplitudes)
std_min_ripple_amp = np.std(ripple_min_amplitudes)
median_min_ripple_amp = np.median(ripple_min_amplitudes)
print("Mean Min Ripple Amplitude: ", mean_min_ripple_amp)
print("Std Min Ripple Amplitude: ", std_min_ripple_amp)
print("Median Min Ripple Amplitude: ", median_min_ripple_amp)

Mean Min Ripple Amplitude:  -6.082431335159767
Std Min Ripple Amplitude:  6.621939347615306
Median Min Ripple Amplitude:  -2.7196811547721467


In [23]:
mean_min_fr_amp = np.mean(fr_min_amplitudes)
std_min_fr_amp = np.std(fr_min_amplitudes)
median_min_fr_amp = np.median(fr_min_amplitudes)
print("Mean Min. FR Amplitude: ", mean_min_fr_amp)
print("Std Min. FR Amplitude: ", std_min_fr_amp)
print("Median Min. FR Amplitude: ", median_min_fr_amp)

Mean Min. FR Amplitude:  -2.8742814904079115
Std Min. FR Amplitude:  2.5811110827463453
Median Min. FR Amplitude:  -1.3973098519987506


In [24]:
mean_min_hfo_amp = np.mean(hfo_min_amplitudes)
std_min_hfo_amp = np.std(hfo_min_amplitudes)
median_min_hfo_amp = np.median(hfo_min_amplitudes)
print("Mean Min. HFO Amplitude: ", mean_min_hfo_amp)
print("Std Min. HFO Amplitude: ", std_min_hfo_amp)
print("Median Min. HFO Amplitude: ", median_min_hfo_amp)

Mean Min. HFO Amplitude:  -6.2874925556412995
Std Min. HFO Amplitude:  6.753113903738274
Median Min. HFO Amplitude:  -3.113791062376728


## 3. Take the mean of the lowest quartile as the baseline amplitude.

In [25]:
#Mean of the lowest quartile (Q1 - 25%)
lowest_quartile_ripple = np.percentile(ripple_max_amplitudes, 25)
lowest_quartile_fr = np.percentile(fr_max_amplitudes, 25)
lowest_quartile_hfo = np.percentile(hfo_max_amplitudes, 25)
# TODO: For now using the DN_Threshold as the symetric value of the UP_THRESHOLD

### Store the Amplitude Thresholds that will be used to convert the signal into spikes

In [26]:
# Store the amplitude thresholds
out_filename = f"{RESULTS_FOLDER}/{DATASET_FILENAME}_thresholds_q1.npy"
q1_output = [lowest_quartile_ripple, lowest_quartile_fr, lowest_quartile_hfo]
print("Q1 Strategy Output:", q1_output)

EXPORT_THRESHOLD_Q1 = True
if EXPORT_THRESHOLD_Q1:
    np.save(out_filename, np.array(q1_output))

Q1 Strategy Output: [2.1719012029537117, 1.173483014880868, 2.5424697548348907]


## 4. Use the mean of the amplitudes array as the baseline amplitude.

In [73]:
# Store the amplitude thresholds
out_filename = f"{RESULTS_FOLDER}/{DATASET_FILENAME}_thresholds_mean.npy"

mean_output = [mean_max_ripple_amp, mean_max_fr_amp, mean_max_hfo_amp]
print("Mean Strategy Output: ", mean_output)

EXPORT_THRESHOLD_MEAN = True
if EXPORT_THRESHOLD_MEAN:
    np.save(out_filename, np.array(mean_output))

Mean Strategy Output:  [6.128352043987768, 2.7920384114064505, 6.317369652901671]


## 5. Use the median of the amplitudes array as the baseline amplitude.

In [74]:
# Store the amplitude thresholds
out_filename = f"{RESULTS_FOLDER}/{DATASET_FILENAME}_thresholds_median.npy"

median_output = [median_max_ripple_amp, median_max_fr_amp, median_max_hfo_amp]
print("Median Strategy Output: ", median_output)

EXPORT_THRESHOLD_MEDIAN = True
if EXPORT_THRESHOLD_MEDIAN:
    np.save(out_filename, np.array(median_output))

Median Strategy Output:  [2.6774136055947038, 1.3753146660939348, 3.0929883901351527]


## 6. Use the Q3 (75 percentile) of the amplitudes array as the baseline amplitude + Scaling Factor

In [27]:
# Mean of the lowest quartile (Q1 - 25%)
highest_quartile_ripple = np.percentile(ripple_max_amplitudes, 75)
highest_quartile_fr = np.percentile(fr_max_amplitudes, 75)
highest_quartile_hfo = np.percentile(hfo_max_amplitudes, 75)
# TODO: For now using the DN_Threshold as the symetric value of the UP_THRESHOLD

In [30]:
# Define scaling factor for the thresholds
ripple_scaling_factor = 0.6
fr_scaling_factor = 0.3
hfo_scaling_factor = 0.45    # TODO: Check the scaling factor for the HFO band

In [31]:
# Store the amplitude thresholds
out_filename = f"{RESULTS_FOLDER}/{DATASET_FILENAME}_thresholds_q3.npy"

q3_output = [highest_quartile_ripple * ripple_scaling_factor, highest_quartile_fr * fr_scaling_factor,
             highest_quartile_hfo * hfo_scaling_factor]
print("Q3 Strategy Output: ", q3_output)

Q3 Strategy Output:  [5.655608080268563, 1.604125245466202, 3.8639991314147633]


In [32]:
EXPORT_THRESHOLD_Q3 = True
if EXPORT_THRESHOLD_Q3:
    np.save(out_filename, np.array(q3_output))

## 7. Use the (60 percentile) of the amplitudes array as the baseline amplitude.

In [78]:
# 60 Percentile
sixty_percentile_ripple = np.percentile(ripple_max_amplitudes, 60)
sixty_percentile_fr = np.percentile(fr_max_amplitudes, 60)
sixty_percentile_hfo = np.percentile(hfo_max_amplitudes, 60)

# Store the amplitude thresholds
out_filename = f"{RESULTS_FOLDER}/{DATASET_FILENAME}_thresholds_60perc.npy"

sixty_perc_output = [sixty_percentile_ripple, sixty_percentile_fr, sixty_percentile_hfo]
print("60% Strategy Output: ", sixty_perc_output)

EXPORT_THRESHOLD_60PERC = True
if EXPORT_THRESHOLD_60PERC:
    np.save(out_filename, np.array(sixty_perc_output))

60% Strategy Output:  [3.10735412276113, 1.4951016536947659, 3.66685555811262]


## 8. Use the (70 percentile) of the amplitudes array as the baseline amplitude.

In [79]:
# 70 Percentile
seventy_percentile_ripple = np.percentile(ripple_max_amplitudes, 70)
seventy_percentile_fr = np.percentile(fr_max_amplitudes, 70)
seventy_percentile_hfo = np.percentile(hfo_max_amplitudes, 70)

# Store the amplitude thresholds
out_filename = f"{RESULTS_FOLDER}/{DATASET_FILENAME}_thresholds_70perc.npy"

seventy_perc_output = [seventy_percentile_ripple, seventy_percentile_fr, seventy_percentile_hfo]
print("70% Strategy Output: ", seventy_perc_output)

EXPORT_THRESHOLD_70PERC = True
if EXPORT_THRESHOLD_70PERC:
    np.save(out_filename, np.array(seventy_perc_output))

70% Strategy Output:  [7.513982800641984, 2.6148472778884853, 7.717717929852056]


## 9. Use the (65 percentile) of the amplitudes array as the baseline amplitude.

In [80]:
# 65 Percentile
sixty_percentile_ripple = np.percentile(ripple_max_amplitudes, 65)
sixty_percentile_fr = np.percentile(fr_max_amplitudes, 65)
sixty_percentile_hfo = np.percentile(hfo_max_amplitudes, 65)

# Store the amplitude thresholds
out_filename = f"{RESULTS_FOLDER}/{DATASET_FILENAME}_thresholds_65perc.npy"

sixty_perc_output = [sixty_percentile_ripple, sixty_percentile_fr, sixty_percentile_hfo]
print("65% Strategy Output: ", sixty_perc_output)

EXPORT_THRESHOLD_65PERC = True
if EXPORT_THRESHOLD_65PERC:
    np.save(out_filename, np.array(sixty_perc_output))

65% Strategy Output:  [4.452286751140795, 1.5912047492515444, 5.255464870113121]
